In [1]:
import numpy as np
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\jw160\project\RAG\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.messages import HumanMessage, AIMessage

In [14]:
from langchain.chat_models import init_chat_model

In [3]:
embeddings = HuggingFaceEndpointEmbeddings(client="http://127.0.0.1:8080/")

In [7]:
import numpy as np
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ==========================================
# 1. 설정 및 초기화 (CPU용 가벼운 임베딩)
# ==========================================
# 로컬에서 무료로 돌아가는 초경량 임베딩 모델 (80MB)
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# 타겟 스키마 (우리가 맞추고자 하는 표준 컬럼들)
TARGET_SCHEMA = ["user_id", "user_name", "phone_number", "email_address", "join_date"]
# 타겟 스키마를 미리 벡터화 (CPU 캐싱 효과)
target_embeddings = embed_model.encode(TARGET_SCHEMA)


In [8]:

# ==========================================
# 2. 상태(State) 정의
# ==========================================
class MappingState(TypedDict):
    source_col: str         # 입력: 분석할 소스 컬럼명 (예: "hp")
    candidates: List[str]   # 중간: 임베딩으로 찾은 후보군
    final_mapping: str      # 결과: SLM이 선택한 최종 매핑
    reasoning: str          # 결과: SLM의 선택 이유

# ==========================================
# 3. 노드(Node) 정의
# ==========================================

def retriever_node(state: MappingState):
    """
    [Stage 1] 임베딩 유사도 검색
    SLM을 쓰지 않고 수학적으로 가장 가까운 후보 3개를 추립니다.
    """
    source_vec = embed_model.encode([state["source_col"]])
    
    # 코사인 유사도 계산
    similarities = cosine_similarity(source_vec, target_embeddings)[0]
    
    # 상위 3개 인덱스 추출
    top_k_indices = np.argsort(similarities)[-3:][::-1]
    candidates = [TARGET_SCHEMA[i] for i in top_k_indices]
    
    print(f"🔍 [Retriever] '{state['source_col']}' 유사 후보: {candidates}")
    return {"candidates": candidates}

def slm_reasoning_node(state: MappingState):
    """
    [Stage 2] SLM 추론 (여기에 somollm/smolagents 연결)
    후보군 중에서 문맥상 가장 적절한 것을 3B 모델이 선택합니다.
    """
    source = state["source_col"]
    candidates = state["candidates"]
    
    # --- 실제로는 여기서 llama.cpp 또는 smolagents를 호출합니다 ---
    # prompt = f"Source 컬럼 '{source}'는 다음 후보 중 무엇과 매핑될까? {candidates}"
    # response = local_llm.invoke(prompt)
    
    # [시뮬레이션] SLM이 'hp'는 'phone_number'라고 판단했다고 가정
    if "hp" in source or "tel" in source:
        selected = "phone_number"
        reason = "hp는 Hand Phone의 약어로 추정됨."
    else:
        selected = candidates[0] # 잘 모르면 1순위 선택
        reason = "가장 유사도가 높음."
    # ----------------------------------------------------------
    
    print(f"🤖 [SLM Agent] 최종 선택: '{selected}' (이유: {reason})")
    return {"final_mapping": selected, "reasoning": reason}


In [15]:
model = init_chat_model(
    "smollm2",
    model_provider="ollama",
    base_url="http://localhost:11434",
    temperature=0,
    max_tokens=126,
    model_kwargs={
    "num_predict": 50  # Ollama 전용 파라미터
    }
)


In [16]:
# [SLM을 쓴다면] -> 고민을 함
def slm_logic(source_col, data_sample, candidates):
    # 데이터 샘플까지 고려하여 판단 요청
    prompt = f"""
    Source Column: {source_col}
    Sample Data: {data_sample} (이 값을 보고 문맥을 파악해)
    Candidates: {candidates}
    
    위 후보 중 데이터 샘플의 성격과 가장 일치하는 컬럼을 골라줘.
    """
    return model.invoke(prompt) # LLM이 '생각'해서 고름

In [10]:
# ==========================================
# 4. LangGraph 그래프 구성
# ==========================================
workflow = StateGraph(MappingState)

# 노드 추가
workflow.add_node("retriever", retriever_node)
workflow.add_node("reasoner", slm_reasoning_node)

# 엣지 연결 (순서 정의)
workflow.set_entry_point("retriever")
workflow.add_edge("retriever", "reasoner")
workflow.add_edge("reasoner", END)

# 컴파일
app = workflow.compile()


In [11]:
# ==========================================
# 5. 실행 테스트
# ==========================================
inputs = {"source_col": "sma"} # 'hp'라는 컬럼이 들어왔을 때
result = app.invoke(inputs)

print("\n✅ 최종 결과:", result["final_mapping"])

🔍 [Retriever] 'sma' 유사 후보: ['email_address', 'user_name', 'phone_number']
🤖 [SLM Agent] 최종 선택: 'email_address' (이유: 가장 유사도가 높음.)

✅ 최종 결과: email_address
